# Kompresja danych
## 1. Kodowanie o stałej długości słowa

- **Jaka jest najkrótsza możliwa długość takiego kodu dla korpusu z dzi-siejszych zajęć?**

Tekst zawiera 37 znaków, co za tym idzie wystarczy z nadmiarem kod o długości 6. Pozwoli to na zakodowanie maksymalnie 2^6 = 64

- **Ile wyniesie stopień kompresji w tym kodzie?**

Stopień kompresji: 1 - 6/8 = 1/4. Plik powinien być mniejszy o 1/4.

In [92]:
from bitarray import bitarray
import operator
import pickle
import os
from collections import Counter
from math import ceil, log2

In [150]:
def load_file(filename):
    with open(filename) as f:
        return f.read()

    
TEST_TEXT_NAME = "text_test.pkl"
TEST_CODE_NAME = "code_test.pkl"

In [143]:
# Ogólny interfejs dla wszystkich metod kodowania

class Coder():
    @staticmethod
    def create(char_counter):
        pass
    
    @staticmethod
    def encode(text, code):
        pass
    
    @staticmethod
    def decode(text_bin, code):
        pass
    
    @staticmethod
    def save(text_enc, code, ftname, fcname):
        with open(ftname, "wb") as ft, open(fcname, "wb") as fc:
            pickle.dump(text_enc, ft)
            pickle.dump(code, fc) 
            
    @staticmethod
    def load(ftname, fcname):
        with open(ftname, "rb") as ft, open(fcname, "rb") as fc:
            text_enc = pickle.load(ft) 
            code = pickle.load(fc)
            
        return text_enc, code

In [144]:
# Kodowanie o stałej długości słowa

class ConstLenCoder(Coder):
    @staticmethod
    def encode(text, code):
        text_bin = [code[letter] for letter in text]
        text_bin = "".join(text_bin)
        text_bin = bitarray(text_bin)
        return text_bin
    
    @staticmethod
    def decode(text_bin, code):
        code_len = len(list(code.values())[0])
        num2letter = {n: l for l, n in code.items()}
        text_bin = [text_bin[i:i+code_len] for i in range(0, len(text_bin), code_len)]
        text = [num2letter[num.to01()] for num in text_bin]
        text = "".join(text)
        return text
    
    @staticmethod
    def create(char_counter):
        code_len = ceil(log2(len(char_counter.keys())))
        code = {char: bin(i)[2:].zfill(code_len) for i, (char, _) in enumerate(char_counter.most_common())}
        return code

In [160]:
# Funckje testujące

def test(coder, text_path):
    text = load_file(text_path)
    
    code = coder.create(Counter(text))
    enc_text = coder.encode(text, code)

    coder.save(enc_text, code, TEST_TEXT_NAME, TEST_CODE_NAME)

    enc_text2, code2 = coder.load(TEST_TEXT_NAME, TEST_CODE_NAME)
    dec_text = coder.decode(enc_text2, code2)
    
    compression = os.path.getsize(TEST_TEXT_NAME) /  os.path.getsize(text_path)
    
    print("Teksty są identyczne:", dec_text == text)
    print("Kompresja: ", compression)

In [162]:
# Test
test(ConstLenCoder, "lab_huffman/norm_wiki_sample.txt")

Teksty są identyczne: True
Kompresja:  0.7500057697970542
